In [2]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 99.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 125.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 80.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 22.9 MB/s eta 0:00:00


In [3]:
import tensorflow as tf### models
import numpy as np### math computations
import matplotlib.pyplot as plt### plotting bar chart
import sklearn### machine learning library
import cv2## image processing
from sklearn.metrics import confusion_matrix, roc_curve### metrics
import seaborn as sns### visualizations
import datetime
import pathlib
import io
import os
import re
import string
import time
from numpy import random
import gensim.downloader as api
from PIL import Image
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Dense,Flatten,InputLayer,BatchNormalization,Dropout,Input,LayerNormalization
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from google.colab import drive
from google.colab import files
from datasets import load_dataset
from transformers import GPT2TokenizerFast,create_optimizer,DataCollatorForLanguageModeling,TFGPT2LMHeadModel

In [4]:
MAX_LENGTH=256
BATCH_SIZE=6

In [5]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d juicobowley/drake-lyrics
!unzip "/content/drake-lyrics.zip" -d "/content/dataset/"

100% 764k/764k [00:00<00:00, 863kB/s]
100% 764k/764k [00:00<00:00, 863kB/s]
Archive:  /content/drake-lyrics.zip
  inflating: /content/dataset/drake_data.csv  
  inflating: /content/dataset/drake_data.json  
  inflating: /content/dataset/drake_lyrics.txt  


In [6]:
filepath="/content/dataset/drake_data.csv"
dataset=load_dataset('csv',data_files=filepath)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-9cc7739a0c963c28/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['album', 'lyrics_title', 'lyrics_url', 'lyrics', 'track_views'],
        num_rows: 290
    })
})

In [8]:
dataset['train'][45]

{'album': 'Scorpion',
 'lyrics_title': 'Emotionless Lyrics',
 'lyrics_url': 'https://genius.com/Drake-emotionless-lyrics',
 'lyrics': '[Chorus: Mariah Carey]\nYou\'ve got me feeling emotions\nAyy, higher\nAhhhhhhh\nYou\'ve got me fe–\nYou, ohhhh\n\n[Verse 1: Drake]\nDon\'t link me\nDon\'t hit me when you hear this and tell me your favorite song\nDon\'t tell me how you knew it would be like this all along\nI know the truth is you won\'t love me until I\'m gone\nAnd even then the thing that comes after is movin\' on\nI can\'t even capture the feeling I had at first\nMeetin\' all my heroes like seein\' how magic works\nThe people I look up to are goin\' from bad to worse\nTheir actions out of character even when they rehearse\nWorkin\' in the land of the free, the home of the brave\nI gotta bring my brothers or else I feel out of place\nBreakin\' speed records on roads that these niggas paved\nAnd they don\'t like that, it\'s written all on they face\nI don\'t know how I\'ma make it out o

In [9]:
model_id="gpt2-medium"
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)

In [10]:
n_wasted=0

In [11]:
for i in range(len(dataset['train'])):
   try:
     outputs = tokenizer(
       dataset["train"][i]["lyrics"],
       truncation=True,
       max_length=256,
       return_overflowing_tokens=True,
       return_length=True,
     )
     print(i,outputs['length'])

     for k in outputs['length']:
       if k!=256:
         n_wasted+=k
   except:
     print('----------------------->i',i)

0 [33]
1 [252]
2 [256, 256, 256, 72]
3 [256, 256, 48]
4 [256, 256, 137]
5 [256, 256, 256, 159]
6 [256, 256, 256, 12]
7 [146]
8 [256, 256, 252]
9 [256, 256, 210]
10 [256, 256, 256, 122]
11 [256, 256, 256, 86]
12 [256, 256, 173]
13 [256, 256, 256, 249]
14 [256, 256, 256, 168]
15 [256, 256, 229]
16 [256, 256, 185]
17 [256, 256, 256, 256, 76]
18 [256, 256, 256, 138]
19 [256, 256, 256, 140]
20 [256, 256, 256, 155]
21 [256, 256, 256, 256, 236]
22 [256, 256, 256, 59]
23 [256, 256, 256, 167]
24 [256, 256, 115]
25 [256, 256, 256, 30]
26 [256, 256, 256, 79]
27 [256, 256, 256, 256, 71]
28 [256, 256, 256, 204]
29 [256, 256, 256, 226]
30 [256, 256, 256, 63]
31 [256, 256, 57]
32 [256, 229]
33 [256, 256, 256, 256, 168]
34 [256, 256, 256, 256, 35]
35 [256, 256, 256, 256, 82]
36 [256, 256, 242]
37 [256, 256, 107]
38 [256, 256, 185]
39 [256, 164]
40 [256, 256, 256, 75]
41 [256, 256, 256, 141]
42 [256, 253]
43 [256, 256, 256, 256, 152]
44 [256, 256, 27]
45 [256, 256, 256, 99]
46 [256, 256, 102]
47 [256, 

In [12]:
def preprocess_function(example):
  try:
    outputs = tokenizer(
        example["lyrics"],
        truncation=True,
        max_length=MAX_LENGTH,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
      if length==MAX_LENGTH:
        input_batch.append(input_ids)
        valid_input_ids=input_ids
    if len(input_batch)!=0:
      for i in range(BATCH_SIZE-len(input_batch)):
        input_batch.append(valid_input_ids)
  except:
    print(example)
    input_batch=[]
  return {"input_ids": input_batch}

In [13]:
tokenized_dataset=dataset.map(
    preprocess_function,remove_columns=dataset["train"].column_names
)

Map:   0%|          | 0/290 [00:00<?, ? examples/s]

{'album': 'Thank Me Later', 'lyrics_title': 'Thank Me Later [Booklet] Lyrics', 'lyrics_url': 'https://genius.com/Drake-thank-me-later-booklet-annotated', 'lyrics': None, 'track_views': '6.2K'}
{'album': 'Unreleased Songs', 'lyrics_title': 'Untitled DaBaby Collaboration* (Ft. DaBaby) Lyrics', 'lyrics_url': 'https://genius.com/Drake-untitled-dababy-collaboration-lyrics', 'lyrics': None, 'track_views': '(Unreleased)'}


In [14]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 290
    })
})

In [15]:
def filter_out(example):
  if len(example['input_ids'])>=1:
    return example

In [16]:
tokenized_full_dataset=tokenized_dataset.filter(filter_out)
print(tokenized_full_dataset)

Filter:   0%|          | 0/290 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 270
    })
})


In [17]:
tokenized_full_dataset['train'][5]

{'input_ids': [[58,
   5317,
   305,
   60,
   198,
   36837,
   319,
   23432,
   1675,
   6733,
   198,
   198,
   58,
   1925,
   15125,
   60,
   198,
   12295,
   550,
   257,
   1256,
   11,
   428,
   318,
   477,
   314,
   761,
   198,
   8061,
   1239,
   1337,
   447,
   227,
   470,
   346,
   447,
   227,
   270,
   338,
   371,
   13,
   40,
   13,
   47,
   198,
   45,
   6950,
   292,
   2900,
   447,
   227,
   9930,
   736,
   319,
   502,
   329,
   645,
   447,
   227,
   11274,
   1738,
   198,
   43,
   4815,
   774,
   318,
   49083,
   290,
   340,
   338,
   477,
   314,
   761,
   198,
   6090,
   470,
   4245,
   257,
   7696,
   655,
   284,
   1657,
   616,
   835,
   198,
   43,
   12375,
   5433,
   319,
   262,
   13956,
   314,
   1101,
   256,
   27048,
   6,
   198,
   47,
   454,
   259,
   6,
   503,
   616,
   5848,
   290,
   340,
   1244,
   2128,
   7165,
   198,
   43,
   12375,
   2121,
   259,
   6,
   12198,
   1037,
   502,
   1382,
   8489

In [18]:
max_batch_len=0

In [19]:
for i in range(270):
  if len(tokenized_full_dataset['train'][i]['input_ids'])>max_batch_len:
    max_batch_len=len(tokenized_full_dataset['train'][i]['input_ids'])
  #print(i,len(tokenized_full_dataset['train'][i]['input_ids']))

In [20]:
print(max_batch_len)

6


In [21]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False, return_tensors="tf")

In [22]:
tf_train_dataset=tokenized_full_dataset['train'].to_tf_dataset(
    columns=["input_ids","attention_mask","labels"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=1,

)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [23]:
for i in tf_train_dataset.take(1):
  print(i)

{'input_ids': <tf.Tensor: shape=(1, 6, 256), dtype=int64, numpy=
array([[[   58,  5317,   305, ...,  1925, 15125,    25],
        [18515,    60,   198, ...,   345,   821,   866],
        [18515,    60,   198, ...,   345,   821,   866],
        [18515,    60,   198, ...,   345,   821,   866],
        [18515,    60,   198, ...,   345,   821,   866],
        [18515,    60,   198, ...,   345,   821,   866]]])>, 'attention_mask': <tf.Tensor: shape=(1, 6), dtype=int64, numpy=array([[1, 1, 1, 1, 1, 1]])>, 'labels': <tf.Tensor: shape=(1, 6, 256), dtype=int64, numpy=
array([[[   58,  5317,   305, ...,  1925, 15125,    25],
        [18515,    60,   198, ...,   345,   821,   866],
        [18515,    60,   198, ...,   345,   821,   866],
        [18515,    60,   198, ...,   345,   821,   866],
        [18515,    60,   198, ...,   345,   821,   866],
        [18515,    60,   198, ...,   345,   821,   866]]])>}


In [24]:
def adjust_attention_mask(input):
  return {'input_ids':input['input_ids'],
          'attention_mask':tf.ones([1,BATCH_SIZE,MAX_LENGTH]),
          'labels':input['labels']}

In [25]:
train_dataset=tf_train_dataset.map(adjust_attention_mask)

In [26]:
for i in train_dataset.take(1):
  print(i)

{'input_ids': <tf.Tensor: shape=(1, 6, 256), dtype=int64, numpy=
array([[[   58, 13414,   325, ..., 20079,  3256,  4308],
        [  198,  3347,   655, ...,   340,   338,   319],
        [  198,  1639,   760, ..., 20917, 12911,   338],
        [13075, 43066,    11, ...,   644,   510,    11],
        [31645,    30,   198, ...,   198, 17140,   502],
        [31645,    30,   198, ...,   198, 17140,   502]]])>, 'attention_mask': <tf.Tensor: shape=(1, 6, 256), dtype=float32, numpy=
array([[[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]]], dtype=float32)>, 'labels': <tf.Tensor: shape=(1, 6, 256), dtype=int64, numpy=
array([[[   58, 13414,   325, ..., 20079,  3256,  4308],
        [  198,  3347,   655, ...,   340,   338,   319],
        [  198,  1639,   760, ..., 20917, 12911,   338],
        [13075, 43066,    11, ...

In [27]:
unbatched_dataset=train_dataset.unbatch()

In [28]:
for i in unbatched_dataset.take(1):
  print(i)

{'input_ids': <tf.Tensor: shape=(6, 256), dtype=int64, numpy=
array([[   58, 13414,   325, ...,   262,  2576,   338],
       [  257,  3288,   198, ..., 11841, 25724,   198],
       [   34,   729,   278, ...,    11,  2553,  1655],
       [   34,   729,   278, ...,    11,  2553,  1655],
       [   34,   729,   278, ...,    11,  2553,  1655],
       [   34,   729,   278, ...,    11,  2553,  1655]])>, 'attention_mask': <tf.Tensor: shape=(6, 256), dtype=float32, numpy=
array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)>, 'labels': <tf.Tensor: shape=(6, 256), dtype=int64, numpy=
array([[   58, 13414,   325, ...,   262,  2576,   338],
       [  257,  3288,   198, ..., 11841, 25724,   198],
       [   34,   729,   278, ...,    11,  2553,  1655],
       [   34,   729,   278, ...,    11,  2553,  1655],
   

In [29]:
model_lyrics = TFGPT2LMHeadModel.from_pretrained(model_id)
model_lyrics.summary()

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2-medium.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLaye  multiple                 354823168 
 r)                                                              
                                                                 
Total params: 354,823,168
Trainable params: 354,823,168
Non-trainable params: 0
_________________________________________________________________


In [30]:
num_train_steps=len(unbatched_dataset)
optimizer, schedule = create_optimizer(
    init_lr=5e-5,
    num_warmup_steps=1_000,
    num_train_steps=num_train_steps,
)
model_lyrics.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [31]:
history=model_lyrics.fit(unbatched_dataset, epochs=5)

Epoch 1/5
270/270 [==============================] - 402s 1s/step - loss: 3.2653
Epoch 2/5
270/270 [==============================] - 334s 1s/step - loss: 2.9754
Epoch 3/5
270/270 [==============================] - 335s 1s/step - loss: 2.7428
Epoch 4/5
270/270 [==============================] - 335s 1s/step - loss: 2.4584
Epoch 5/5
270/270 [==============================] - 335s 1s/step - loss: 2.2051


In [32]:
from google.colab import drive
drive.mount('/content/drive')

# Save the model weights to a file on Google Drive
model_lyrics.save_weights('/content/drive/MyDrive/my_model_lyrics_weights.h5')


Mounted at /content/drive


In [33]:
model_lyrics.load_weights('/content/drive/MyDrive/my_model_lyrics_weights.h5')

In [34]:
input_text="true love shouldn't be this complicated"

In [35]:
input_ids = tokenizer(input_text, return_tensors="tf")["input_ids"]

In [36]:
init_time=time.time()
output_greedy = model_lyrics.generate(input_ids,max_length=256,do_sample=False)
print(tokenizer.decode(output_greedy[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


true love shouldn't be this complicated
I'm just trying to be myself
I'm just trying to be myself
I'm just trying to be myself
I'm just trying to be myself

[Chorus]
I'm just trying to be myself
I'm just trying to be myself
I'm just trying to be myself
I'm just trying to be myself
I'm just trying to be myself

[Verse 2]
I'm just trying to be myself
I'm just trying to be myself
I'm just trying to be myself
I'm just trying to be myself
I'm just trying to be myself

[Pre-Chorus]
I'm just trying to be myself
I'm just trying to be myself
I'm just trying to be myself
I'm just trying to be myself

[Chorus]
I'm just trying to be myself
I'm just trying to be myself
I'm just trying to be myself
I'm just trying to be myself
I'm just trying to be myself

[Verse 3]
I'm just trying to be myself
I'm just trying to be myself
I'm just trying to be myself
I'm just trying to be
154.50581645965576


In [ ]:
init_time=time.time()
output_beam = model_lyrics.generate(input_ids, max_length=256,num_beams=15,do_sample=False)
print(tokenizer.decode(output_beam[0]))
print(time.time()-init_time)

In [42]:
init_time=time.time()
output_temp = model_lyrics.generate(input_ids, max_length=256, do_sample=True,temperature=1.0, top_k=20)
print(tokenizer.decode(output_temp[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


true love shouldn't be this complicated
I can't do this shit alone
They need to know they heard me say it

[Chorus: Rihanna & Drake]
I know you love me for who I am
So what's next? Are we on tour?
You just told me that it's over
Can't do this shit alone
We have to find another

[Interlude: Rihanna]
You can't fake that love for me, no, no, no
I know you love me for who I am
So what's next? Are we on tour?
You just told me that it's over
Can't do this shit alone

We have to find another

[Verse 2: Drake & Rihanna]
You say you love me
What are you getting at with these words?
What are they going for and what do they mean?
I was never meant to be this way
So what's next? Are we on tour?
You just told me that it's over
Can't do this shit alone

We have to find another

[Chorus: Rihanna & Drake]
I know you love me for who I am
So what's next
123.43479943275452


In [44]:
init_time=time.time()
output_temp = model_lyrics.generate(input_ids, max_length=256, do_sample=True,temperature=0.5, top_k=0)
print(tokenizer.decode(output_temp[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


true love shouldn't be this complicated
He said, "I don't know, girl, I guess I just love you"
I said, "No, no, no, no, no, no"
I know, I know, I know, I know, I know

[Chorus: Drake]
I love you, I love you, I love you
I love you, I love you, I love you
I love you, I love you, I love you
I love you, I love you, I love you
I love you, I love you, I love you
I love you, I love you, I love you
I love you, I love you, I love you

[Verse 2: Young Thug]
I'ma take you down to a club and then you can take me there
If you wanna fuck, then I'll take it there
But you don't have to say yes
I'll take it there, I'll take it there
But you don't have to say yes
I'll take it there, I'll take it there
But you don't have to say yes
I'll take it there, I'll take it there
But you
94.33288431167603


In [43]:
init_time=time.time()
output_temp = model_lyrics.generate(input_ids, max_length=256, do_sample=True,temperature=0.5, top_k=0)
print(tokenizer.decode(output_temp[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


true love shouldn't be this complicated

[Verse 2]
I'm just a regular girl, I can't do no wrong
And I'm just a regular girl, I can't do no wrong
I've been around so many different people
I don't know what to do with myself
I don't know what to do with myself
I don't know what to do with myself
I don't know what to do with myself
I just can't do no wrong

[Chorus]
I just can't do no wrong
I just can't do no wrong
I just can't do no wrong
I just can't do no wrong
I just can't do no wrong

[Verse 3]
I don't know what to do with myself
I just can't do no wrong
I just can't do no wrong
I just can't do no wrong

[Chorus]
I just can't do no wrong
I just can't do no wrong
I just can't do no wrong
I just can't do no wrong

[Verse 4]
I just can't do no wrong
I just can't do no wrong
I just can't do no wrong
96.33770895004272


In [46]:
init_time=time.time()
output_topk = model_lyrics.generate(input_ids, max_length=256, do_sample=True,temperature=2.0,top_k=50)
print(tokenizer.decode(output_topk[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


true love shouldn't be this complicated"
I hope we're on a song that only two of the women can say; "Look at it through the lenses of love " (Ayy)
The camera gets blurry
In the back of somebody's mind where they don't even know they feel
It just starts to freak a nigga upside to try get a fix down like,

[Verse 6]
Nigga just got your ass at Milly
So just watch how these hoes pull out tricks off you
Even try and tell a ghost ghost story that turned nasty
All these girls that your dreams wanna save can't keep ya alone (Won't ya),
Cats love ya' but the dogs still adore ya

[Ayy 2]
Get in the bed where no one dows in
Start off with something 'round 'fo' somebody
Like twenty drinks for ten that make a nigga's world fade
The things they think they know don't have you toin'
Cinderella
Veg and make a lot of mistakes to be the last girl
That puts this fuck and girl right now with ya dick
I did this shit to get an honest sound and then
A real nig
99.45424509048462


In [47]:
init_time=time.time()
output_topp = model_lyrics.generate(input_ids, max_length=256, do_sample=True,top_p=0.90)
print(tokenizer.decode(output_topp[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


true love shouldn't be this complicated"
And if that's not enough for you
Then don't hesitate to text me
I'm at home, I gotta take a break
And then I'll be back on top
See you soon
<|endoftext|>
17.33092761039734
